In [ ]:
#| export
#| default_exp dhp

In [ ]:
#| export
#| export
doc = """**Dialog Helper for Polya's Problem-Solving Method**

This module provides quick access to Polya's four-stage problem-solving process
through interactive prompts.

Each stage has multiple questions/prompts:
- To preview one you can just print it/type its name in a cell and hit Submit
  - e.g. `dhp.act.next` shows you "(prompt) What is next?".
- To execute one you call it by adding () after the name
  - e.g. `dhp.act.next()` will replace the current message cell with a prompt cell having "What is next?" in it
    - It will be automatically executed, you can hit Esc to stop it and/or Enter to edit the prompt

**TYPICAL FLOW:**
- Start with `dhp.u` (`understand`) (even briefly) to clarify your understanding of the problem
- Move to `dhp.p` (`plan`) to develop initial strategies
- Switch between `plan` and `act` (or `a`, or `x`/`execute`) as you develop and test approaches
- Use `dhp.r` (`review`) to gain deeper understanding of your approach and findings
- You might loop back to other steps after `r` (`review`)

If you feel stuck, run `dhp.help()` and it will submit the prompt cell it creates - SolveIt will help you pick a next prompt!

**UNDERSTAND STAGE - Clarify the problem before solving**
- dhp.u.summary() - Creates a prompt cell asking SolveIt to give a **concise summary** of the problem
- dhp.u.info() - Creates a prompt cell asking SolveIt to inventory **known/unknown information**
- dhp.u.similar() - Creates a prompt cell asking SolveIt if it has **seen a similar problem** before
- dhp.u.lateral() - Creates a prompt cell to **explore problem relationships and scope**
- dhp.u.related() - Creates a prompt cell to **identify similar or simpler problems**
- dhp.u.viz() - Creates a prompt cell asking SolveIt to **create a figure or diagram** to represent the problem
- dhp.u.notation() - Creates a prompt cell asking SolveIt to **pick suitable notation** (symbols for quantities/data, states, transitions)
- dhp.u.simplest() - Creates a prompt cell asking SolveIt for the **simplest way to look at** the problem
- dhp.u.simplify() - Creates a prompt cell asking SolveIt to **separate problem parts** (break down complex conditions into simpler ones)

**PLAN STAGE - Develop strategies and approaches**
- dhp.p.chunks() - Creates a prompt cell asking SolveIt to **break down the problem** into smaller sub-problems
- dhp.p.partial() - Creates a prompt cell asking SolveIt if there's a **smaller part or representation** of the problem to solve
- dhp.p.known_approach() - Creates a prompt cell asking SolveIt to **use a known algorithm or library** to solve the problem
- dhp.p.verifiable() - Creates a prompt cell asking SolveIt **how to verify** if the solution is consistent and correct
- dhp.p.backward() - Creates a prompt cell asking SolveIt to **work backward** from the desired result
- dhp.p.aux() - Creates a prompt cell asking SolveIt to **use an auxiliary element** (variable, diagram, or example) to clarify the path
- dhp.p.analogy() - Creates a prompt cell asking SolveIt to **use analogy or similarity** to relate the problem to a known solution
- dhp.p.review() - Creates a prompt cell asking SolveIt to **critique the plan of attack** (be frank and critical)

**ACT STAGE - Execute your plan while monitoring progress**
- dhp.a.all() - Creates a prompt cell asking SolveIt if we **covered all of the data or examples** for this step
- dhp.a.check() - Creates a prompt cell asking SolveIt if **this step seems correct**
- dhp.a.doubt() - Creates a prompt cell asking SolveIt if we're **using the right approach**
- dhp.a.next() - Creates a prompt cell asking SolveIt **what is next**
- dhp.a.other() - Creates a prompt cell asking SolveIt if there's **another way to look at this**
- dhp.a.partial() - Creates a prompt cell asking SolveIt about **intermediate results or milestones** to aim for
- dhp.a.simpler() - Creates a prompt cell asking SolveIt if there was a **simpler way** to do this step
- dhp.a.symmetry() - Creates a prompt cell asking SolveIt about **symmetries or patterns** in the problem to exploit
- dhp.a.valid() - Creates a prompt cell asking SolveIt if **this step was a valid step**

**REVIEW STAGE - Verify results, reflect on process, and extract lessons**
- dhp.r.all() - Creates a prompt cell asking SolveIt if we **covered all of the data or examples** for this problem
- dhp.r.alter() - Creates a prompt cell asking SolveIt for **alternative solutions or approaches** that might be more efficient or effective
- dhp.r.general() - Creates a prompt cell asking SolveIt if we can **generalize the solution** to other similar problems
- dhp.r.grok() - Creates a note cell with the text "**To consider:** Can I understand the solution without having to perform all the steps?"
- dhp.r.learned() - Creates a prompt cell asking SolveIt **what lessons have been learned** from this
- dhp.r.mistakes() - Creates a prompt cell asking SolveIt about **common mistakes** made
- dhp.r.other() - Creates a prompt cell asking SolveIt if we can **derive the result differently**
- dhp.r.principles() - Creates a prompt cell asking SolveIt to **identify underlying principles or patterns** that emerged during the solution process
- dhp.r.sanity() - Creates a prompt cell asking SolveIt if the **result makes sense** and can be verified by substitution or another method
- dhp.r.simpler() - Creates a prompt cell asking SolveIt if we can **derive the result in a simpler way**
- dhp.r.test() - Creates a prompt cell asking SolveIt for **different ways to test** this
"""

__all__ = ['u', 'p', 'a', 'r', 'doc', 'understand', 'plan', 'act', 'execute', 'x', 'review', 'help', 'Step', 'QQ', 'PolyaUnderstand', 'PolyaPlan', 'PolyaAct', 'PolyaReview']

from dialoghelper.core import *

# Step: inspired by contextpack.Topic
class Step:
    def _children(self):
        return [c for c in dir(self) if not (c.startswith('_') or c in ('get','getter','docs'))]
    def __iter__(self): 
        yield from (getattr(self, name) for name in self._children() 
                    if isinstance(getattr(self, name),QQ))
    def __repr__(self):
        s = ''
        if self.__doc__: s += f"doc: {self.__doc__}\n"
        if (cs := self._children()): s += "\n".join([c + ": " + repr(getattr(self, c)) for c in cs])
        return s

# QQ replaces the SolveIt cell with its given question/prompt/note
class QQ:
    def __init__(self, content:str, msg_type:str="prompt", run:bool=True):
        self.content = content
        self.msg_type = msg_type
        self.run = run

    def __call__(self):
        _msgid = read_msg(0)['msg']['id']
        update_msg(content=self.content, msg_type=self.msg_type, msgid=_msgid)
        if self.run: run_msg(msgid=_msgid)
        
    def __repr__(self):
        return f"({self.msg_type}) \"{self.content}\""


class PolyaUnderstand(Step):
    """UNDERSTAND STAGE - Clarify the problem before solving
----
- dhp.u.summary() - Creates a prompt cell asking SolveIt to give a **concise summary** of the problem
- dhp.u.info() - Creates a prompt cell asking SolveIt to inventory **known/unknown information**
- dhp.u.similar() - Creates a prompt cell asking SolveIt if it has **seen a similar problem** before
- dhp.u.lateral() - Creates a prompt cell to **explore problem relationships and scope**
- dhp.u.related() - Creates a prompt cell to **identify similar or simpler problems**
- dhp.u.viz() - Creates a prompt cell asking SolveIt to **create a figure or diagram** to represent the problem
- dhp.u.notation() - Creates a prompt cell asking SolveIt to **pick suitable notation** (symbols for quantities/data, states, transitions)
- dhp.u.simplest() - Creates a prompt cell asking SolveIt for the **simplest way to look at** the problem
- dhp.u.simplify() - Creates a prompt cell asking SolveIt to **separate problem parts** (break down complex conditions into simpler ones)
"""
    def __init__(self):
        self.summary = QQ("Could you please give a concise summary of the problem?")
        self.info = QQ("What information do we have? What information do we not have? What might change as we learn more?")
        self.similar = QQ("Have you seen a similar problem before?")
        self.lateral = QQ("Can you relate this problem to a more general or a more specific problem?")
        self.related = QQ("Can you think of a related problem that we can solve? It could even be a simpler one we could solve first to help understand this one.")
        self.viz = QQ("Can we create a figure or diagram to represent the problem?")
        self.notation = QQ("Can we pick a suitable notation (e.g. symbols for quantities/data, states, and transitions)?")        
        self.simplest = QQ("What might be the simplest way to look at this problem?")        
        self.simplify = QQ("Can we separate the various parts of the problem (e.g. break down complex conditions into simpler ones)?")
        

class PolyaPlan(Step):
    """PLAN STAGE - Develop strategies and approaches
----
- dhp.p.chunks() - Creates a prompt cell asking SolveIt to **break down the problem** into smaller sub-problems
- dhp.p.partial() - Creates a prompt cell asking SolveIt if there's a **smaller part or representation** of the problem to solve
- dhp.p.known_approach() - Creates a prompt cell asking SolveIt to **use a known algorithm or library** to solve the problem
- dhp.p.verifiable() - Creates a prompt cell asking SolveIt **how to verify** if the solution is consistent and correct
- dhp.p.backward() - Creates a prompt cell asking SolveIt to **work backward** from the desired result
- dhp.p.aux() - Creates a prompt cell asking SolveIt to **use an auxiliary element** (variable, diagram, or example) to clarify the path
- dhp.p.analogy() - Creates a prompt cell asking SolveIt to **use analogy or similarity** to relate the problem to a known solution
- dhp.p.review() - Creates a prompt cell asking SolveIt to **critique the plan of attack** (be frank and critical)
"""
    def __init__(self):
        self.chunks = QQ("Could you please break down the problem into smaller sub-problems?")
        self.partial = QQ("Is there a smaller part or representation of the problem we could solve?")
        self.known_approach = QQ("Could we use a known algorithm or library to solve the problem, or some of it?")
        self.verifiable = QQ("How would we verify if our solution is consistent and correct?")
        self.backward = QQ("Can we work backward from the desired result?")
        self.aux = QQ("Could we use an auxiliary element (e.g., a variable, diagram, or example) to clarify the path?")
        self.analogy = QQ("Can you use analogy or similarity to relate the problem to a known solution?")
        self.review = QQ("Could you please critique the plan of attack? Be frank, do not be afraid to be critical.")


class PolyaAct(Step):
    """ACT STAGE - Execute your plan while monitoring progress
----
- dhp.a.doubt() - Creates a prompt cell asking SolveIt if we're **using the right approach**
- dhp.a.other() - Creates a prompt cell asking SolveIt if there's **another way to look at this**
- dhp.a.partial() - Creates a prompt cell asking SolveIt about **intermediate results or milestones** to aim for
- dhp.a.symmetry() - Creates a prompt cell asking SolveIt about **symmetries or patterns** in the problem to exploit
- dhp.a.next() - Creates a prompt cell asking SolveIt **what is next**
- dhp.a.valid() - Creates a prompt cell asking SolveIt if **this step was a valid step**
- dhp.a.check() - Creates a prompt cell asking SolveIt if **this step seems correct**
- dhp.a.simpler() - Creates a prompt cell asking SolveIt if there was a **simpler way** to do this step
- dhp.a.all() - Creates a prompt cell asking SolveIt if we **covered all of the data or examples** for this step
"""
    def __init__(self):
        self.doubt = QQ("Are we using the right approach?")
        self.other = QQ("Is there another way to look at this?")
        self.partial = QQ("Are there any intermediate results or milestones that we can aim for?")
        self.symmetry = QQ("Are there any symmetries or patterns in the problem that we can exploit?")
        self.next = QQ("What is next?")
        self.valid = QQ("Does this step seem to have been a valid step?")
        self.check = QQ("Does this step seem correct?")
        self.simpler = QQ("Was there a simpler way we could have done this step?")
        self.all = QQ("Did we cover all of the data or examples for this step?")
        

class PolyaReview(Step):
    """REVIEW STAGE - Verify results, reflect on process, and extract lessons
----
- dhp.r.all() - Creates a prompt cell asking SolveIt if we **covered all of the data or examples** for this problem
- dhp.r.sanity() - Creates a prompt cell asking SolveIt if the **result makes sense** and can be verified by substitution or another method
- dhp.r.grok() - Creates a note cell with the text "**To consider:** Can I understand the solution without having to perform all the steps?"
- dhp.r.learned() - Creates a prompt cell asking SolveIt **what lessons have been learned** from this
- dhp.r.general() - Creates a prompt cell asking SolveIt if we can **generalize the solution** to other similar problems
- dhp.r.alter() - Creates a prompt cell asking SolveIt for **alternative solutions or approaches** that might be more efficient or effective
- dhp.r.other() - Creates a prompt cell asking SolveIt if we can **derive the result differently**
- dhp.r.mistakes() - Creates a prompt cell asking SolveIt about **common mistakes** made
- dhp.r.simpler() - Creates a prompt cell asking SolveIt if we can **derive the result in a simpler way**
- dhp.r.principles() - Creates a prompt cell asking SolveIt to **identify underlying principles or patterns** that emerged during the solution process
- dhp.r.test() - Creates a prompt cell asking SolveIt for **different ways to test** this
"""
    def __init__(self):
        self.all = QQ("Did we cover all of the data or examples for this problem?")
        self.sanity = QQ("Does the result make sense? Can we verify by substition or another method?")
        self.grok = QQ("**To consider:** Can I understand the solution without having to perform all the steps?", "note")
        self.learned = QQ("What lessons have I learned from this?")
        self.general = QQ("Can we generalize the solution to other similar problems?")
        self.alter = QQ("Can you think of alternative solutions or approaches that might be more efficient or effective?")
        self.other = QQ("Can we derive the result differently?")
        self.mistakes = QQ("What were my common mistakes?")
        self.simpler = QQ("Can we derive the result in a simpler way?")
        self.principles = QQ("Can you identify any underlying principles or patterns that emerged during the solution process?")
        self.test = QQ("What are some different ways we can test this?")

u = understand = PolyaUnderstand()
p = plan = PolyaPlan()
x = execute = a = act = PolyaAct()
r = review = PolyaReview()

def overview():
    return "\n".join([f"{repr(globals()[x])}\n" for x in __all__ if x[0].islower() and x not in ('help', 'overview', 'act', 'x', 'execute', 'plan', 'understand', 'review')])
    
def help():
    QQ(f"Please pick an appropriate next-step/prompt from the below:\n\n{overview()}")()